In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

ModuleNotFoundError: No module named 'tensorflow'

In [4]:
print(tf.__version__)

NameError: name 'tf' is not defined

In [13]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [14]:
titanic = pd.read_csv(r"C:\Users\Sub\CodeSpace\AI\dnn-python-from-scratch\final_dataset_ready.csv")
dataset = titanic.copy()
X = np.asarray(dataset.drop('Survived',axis=1))
y = np.asarray(dataset['Survived'])

In [15]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,shuffle=True,random_state=0)

In [16]:
sc = StandardScaler()
X_trainNormalized = sc.fit_transform(X_train)
X_testNormalized = sc.transform(X_test)

In [17]:
numDims = X_trainNormalized.shape[1]
dnnModel = tf.keras.Sequential([
    layers.Dense(16,activation='relu',input_shape=(numDims,)),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(16,activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(126,activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(160,activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(10,activation='relu'),
    layers.Dropout(0.2),
    layers.BatchNormalization(),
    layers.Dense(2)
])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
dnnModel.compile(optimizer=optimizer,loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['Accuracy'])

In [18]:
dnnModel.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 16)                160       
                                                                 
 batch_normalization_5 (Batc  (None, 16)               64        
 hNormalization)                                                 
                                                                 
 dropout_5 (Dropout)         (None, 16)                0         
                                                                 
 dense_7 (Dense)             (None, 16)                272       
                                                                 
 batch_normalization_6 (Batc  (None, 16)               64        
 hNormalization)                                                 
                                                                 
 dropout_6 (Dropout)         (None, 16)               

In [ ]:
dnnModel.fit(X_trainNormalized,y_train,epochs=1000)

Epoch 1/1000
23/23 [==============================] - 0s 9ms/step - loss: 0.4521 - Accuracy: 0.8146
Epoch 2/1000
23/23 [==============================] - 0s 6ms/step - loss: 0.4803 - Accuracy: 0.8034
Epoch 3/1000
23/23 [==============================] - 0s 4ms/step - loss: 0.4595 - Accuracy: 0.8216
Epoch 4/1000
23/23 [==============================] - 0s 3ms/step - loss: 0.4380 - Accuracy: 0.8062
Epoch 5/1000
23/23 [==============================] - 0s 4ms/step - loss: 0.4229 - Accuracy: 0.8076
Epoch 6/1000
23/23 [==============================] - 0s 4ms/step - loss: 0.4517 - Accuracy: 0.8034
Epoch 7/1000
23/23 [==============================] - 0s 4ms/step - loss: 0.4418 - Accuracy: 0.8104
Epoch 8/1000
23/23 [==============================] - 0s 4ms/step - loss: 0.4568 - Accuracy: 0.8104
Epoch 9/1000
23/23 [==============================] - 0s 4ms/step - loss: 0.4428 - Accuracy: 0.8062
Epoch 10/1000
23/23 [==============================] - 0s 4ms/step - loss: 0.4665 - Accuracy: 0.7935

In [10]:
test_loss , test_acc = dnnModel.evaluate(X_testNormalized,y_test,verbose=2)
print('\nTest Accuracy :',test_acc)

6/6 - 2s - 303ms/step - Accuracy: 0.8101 - loss: 0.4190

Test Accuracy : 0.8100558519363403


In [11]:
y_pred = np.argmax(dnnModel.predict(X_test), axis=-1)

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step


In [12]:
idx = np.random.randint(1,len(y_test))
print('Index : ',idx,'Actual value : ',y_test[idx],'Predicted : ',y_pred[idx])

Index :  82 Actual value :  0 Predicted :  0


In [13]:
dnnModel.predict(X_test).shape

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 840us/step


(179, 2)